In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import time
from datetime import datetime
import itertools
import gc
import pickle
from xgboost import XGBRegressor
from xgboost import plot_importance

%matplotlib inline

In [7]:
sales = pd.read_csv('./data/sales_train.csv')
items = pd.read_csv('./data/items.csv')
item_categories = pd.read_csv('./data/item_categories.csv')
shops = pd.read_csv('./data/shops.csv')
test = pd.read_csv('./data/test.csv')

sales1 = pd.read_csv('./data/sales1.csv')
#matrix = pd.read_csv('matrix.csv')

In [8]:
sales1.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-01-02,0,59,22154,999.00,1.0
1,2013-01-03,0,25,2552,899.00,1.0
2,2013-01-06,0,25,2554,1709.05,1.0
3,2013-01-15,0,25,2555,1099.00,1.0
4,2013-01-10,0,25,2564,349.00,1.0


In [9]:
sales1['revenue'] = sales1['item_price'] * sales1['item_cnt_day']
sales1.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,revenue
0,2013-01-02,0,59,22154,999.00,1.0,999.00
1,2013-01-03,0,25,2552,899.00,1.0,899.00
2,2013-01-06,0,25,2554,1709.05,1.0,1709.05
3,2013-01-15,0,25,2555,1099.00,1.0,1099.00
4,2013-01-10,0,25,2564,349.00,1.0,349.00


In [10]:
matrix.head()

NameError: name 'matrix' is not defined

- 월별 판매갯수(item_cnt_month) 데이터 생성 및 matrix merge

In [11]:
group = sales.groupby(['date_block_num', 'shop_id', 'item_id']).agg({'item_cnt_day': ['sum']})
group.head()

item_cnt_day
                                        sum
date_block_num shop_id item_id             
0              0       32               6.0
                       33               3.0
                       35               1.0
                       43               1.0
                       51               2.0

In [12]:
group.columns = ['item_cnt_month']
group.reset_index(inplace=True)
group.head()

,date_block_num,shop_id,item_id,item_cnt_month
0,0,0,32,6.0
1,0,0,33,3.0
2,0,0,35,1.0
3,0,0,43,1.0
4,0,0,51,2.0


In [13]:
# matrix 에 gorup데이터 merge 하여 컬럼을 붙임('item_cnt_month')
matrix = pd.merge(matrix, group, how='left')
matrix.head()
# NaN 값은 판매 개수가 없다고 판단 -> 0으로 처리??

NameError: name 'matrix' is not defined

In [14]:
# 결측값을 어떻게 처리 할 지 생각해보기!!!
# 일단 0으로 처리
matrix['item_cnt_month'] = (matrix['item_cnt_month'].fillna(0).astype(np.float16))
matrix.head()

NameError: name 'matrix' is not defined

In [15]:
matrix1 = matrix.copy()
matrix1.head()

NameError: name 'matrix' is not defined

In [16]:
matrix1.info()

NameError: name 'matrix1' is not defined

In [ ]:
matrix.tail()

In [ ]:
matrix1['date_block_num'] = matrix1['date_block_num'].astype(np.int8)
matrix1['shop_id'] = matrix1['shop_id'].astype(np.int8)
matrix1['item_id'] = matrix1['item_id'].astype(np.int16)
matrix1['ID'] = matrix1['ID'].astype(np.int32)
matrix1['city_id'] = matrix1['city_id'].astype(np.int8)
matrix1['item_category_id'] = matrix1['item_category_id'].astype(np.int8)
matrix1['type_id'] = matrix1['type_id'].astype(np.int8)
matrix1['type_id'] = matrix1['type_id'].astype(np.int8)

In [ ]:
matrix1.info()

### Matrix에 Target lags 추가
item_cnt_month_i : i달 전의 동일 shop, item의 판매량

In [ ]:
def lag_feature(df, lags, col):
    tmp = df[['date_block_num','shop_id','item_id',col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num','shop_id','item_id', col+'_lag_'+str(i)]
        shifted['date_block_num'] += i
        df = pd.merge(df, shifted, on=['date_block_num','shop_id','item_id'], how='left')
    return df

In [ ]:
matrix1 = lag_feature(matrix1, [1,2,3,6,12], 'item_cnt_month')
matrix1

- date_avg_item_cnt(월 평균 판매수량) 컬럼 생성 및 matrix에 추가

In [ ]:
group = matrix1.groupby(['date_block_num']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_avg_item_cnt']
group.reset_index(inplace=True)
group.head()

In [ ]:
matrix1 = pd.merge(matrix1, group, on=['date_block_num'], how='left')
matrix1['date_avg_item_cnt'] = matrix1['date_avg_item_cnt'].astype(np.float16)
matrix1.head()

In [ ]:
matrix2 = matrix1.copy()
matrix2.head()

In [3]:
# matrix에 월 평균 판매 수량에 대한 lag 추가
matrix2 = lag_feature(matrix2, [1], 'date_avg_item_cnt')
matrix2.head()

NameError: name 'lag_feature' is not defined

In [4]:
matrix2.drop(['date_avg_item_cnt'], axis=1, inplace=True)
matrix2.tail()

NameError: name 'matrix2' is not defined

In [40]:
matrix3 = matrix2.copy()
matrix3.head()

,date_block_num,shop_id,item_id,ID,city_id,item_category_id,type_id,item_cnt_month,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3,item_cnt_month_lag_6,item_cnt_month_lag_12,date_avg_item_cnt_lag_1
0,0,0,19,0,29,40,11,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,27,0,29,19,5,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,28,0,29,30,8,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,29,0,29,23,5,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0,32,0,29,40,11,6.0,NaN,NaN,NaN,NaN,NaN,NaN


- date_item_avg_item_cnt(월별 item별 평균 판매수량) 컬럼 생성 및 matrix에 추가

In [41]:
group = matrix3.groupby(['date_block_num', 'item_id']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_item_avg_item_cnt']
group.reset_index(inplace=True)

matrix3 = pd.merge(matrix3, group, on=['date_block_num', 'item_id'], how='left')
matrix3['date_item_avg_item_cnt'] = matrix3['date_item_avg_item_cnt'].astype(np.float16)
matrix3.head()

,date_block_num,shop_id,item_id,ID,city_id,item_category_id,type_id,item_cnt_month,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3,item_cnt_month_lag_6,item_cnt_month_lag_12,date_avg_item_cnt_lag_1,date_item_avg_item_cnt
0,0,0,19,0,29,40,11,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.022217
1,0,0,27,0,29,19,5,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.155518
2,0,0,28,0,29,30,8,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.177734
3,0,0,29,0,29,23,5,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.088867
4,0,0,32,0,29,40,11,6.0,NaN,NaN,NaN,NaN,NaN,NaN,6.644531


In [42]:
matrix3 = lag_feature(matrix3, [1,2,3,6,12], 'date_item_avg_item_cnt')
matrix3.drop(['date_item_avg_item_cnt'], axis=1, inplace=True)
matrix3.tail()

,date_block_num,shop_id,item_id,ID,city_id,item_category_id,type_id,item_cnt_month,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3,item_cnt_month_lag_6,item_cnt_month_lag_12,date_avg_item_cnt_lag_1,date_item_avg_item_cnt_lag_1,date_item_avg_item_cnt_lag_2,date_item_avg_item_cnt_lag_3,date_item_avg_item_cnt_lag_6,date_item_avg_item_cnt_lag_12
11127951,34,45,18454,214195,20,55,13,0.0,1.0,0.0,0.0,0.0,NaN,0.29834,0.045441,0.023254,0.071411,0.590820,NaN
11127952,34,45,16188,214196,20,64,14,0.0,0.0,0.0,NaN,NaN,NaN,0.29834,0.022720,0.069763,NaN,NaN,NaN
11127953,34,45,15757,214197,20,55,13,0.0,0.0,0.0,0.0,0.0,0.0,0.29834,0.113647,0.069763,0.095215,0.250000,0.180054
11127954,34,45,19648,214198,20,40,11,0.0,0.0,0.0,0.0,0.0,NaN,0.29834,0.045441,0.069763,0.166626,0.090881,NaN
11127955,34,45,969,214199,20,37,11,0.0,0.0,0.0,0.0,0.0,0.0,0.29834,0.068176,0.116272,0.023804,0.068176,0.119995


In [43]:
matrix4 = matrix3.copy()
matrix4.head()

,date_block_num,shop_id,item_id,ID,city_id,item_category_id,type_id,item_cnt_month,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3,item_cnt_month_lag_6,item_cnt_month_lag_12,date_avg_item_cnt_lag_1,date_item_avg_item_cnt_lag_1,date_item_avg_item_cnt_lag_2,date_item_avg_item_cnt_lag_3,date_item_avg_item_cnt_lag_6,date_item_avg_item_cnt_lag_12
0,0,0,19,0,29,40,11,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,27,0,29,19,5,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,28,0,29,30,8,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,29,0,29,23,5,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0,32,0,29,40,11,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
# matrix4.to_csv('matrix4.csv', header=True, index=False)

- matrix에 월별, shop_id별 평균 판매 수량(date_shop_avg_item_cnt) lag 추가하기

In [45]:
group = matrix4.groupby(['date_block_num', 'shop_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_shop_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix4 = pd.merge(matrix4, group, on=['date_block_num','shop_id'], how='left')
matrix4['date_shop_avg_item_cnt'] = matrix4['date_shop_avg_item_cnt'].astype(np.float16)

matrix4 = lag_feature(matrix4, [1,2,3,6,12], 'date_shop_avg_item_cnt')
matrix4.drop(['date_shop_avg_item_cnt'], axis=1, inplace=True)
matrix4.tail()

,date_block_num,shop_id,item_id,ID,city_id,item_category_id,type_id,item_cnt_month,item_cnt_month_lag_1,item_cnt_month_lag_2,...,date_item_avg_item_cnt_lag_1,date_item_avg_item_cnt_lag_2,date_item_avg_item_cnt_lag_3,date_item_avg_item_cnt_lag_6,date_item_avg_item_cnt_lag_12,date_shop_avg_item_cnt_lag_1,date_shop_avg_item_cnt_lag_2,date_shop_avg_item_cnt_lag_3,date_shop_avg_item_cnt_lag_6,date_shop_avg_item_cnt_lag_12
11127951,34,45,18454,214195,20,55,13,0.0,1.0,0.0,...,0.045441,0.023254,0.071411,0.590820,NaN,0.129639,0.128662,0.139038,0.144287,NaN
11127952,34,45,16188,214196,20,64,14,0.0,0.0,0.0,...,0.022720,0.069763,NaN,NaN,NaN,0.129639,0.128662,NaN,NaN,NaN
11127953,34,45,15757,214197,20,55,13,0.0,0.0,0.0,...,0.113647,0.069763,0.095215,0.250000,0.180054,0.129639,0.128662,0.139038,0.144287,0.197876
11127954,34,45,19648,214198,20,40,11,0.0,0.0,0.0,...,0.045441,0.069763,0.166626,0.090881,NaN,0.129639,0.128662,0.139038,0.144287,NaN
11127955,34,45,969,214199,20,37,11,0.0,0.0,0.0,...,0.068176,0.116272,0.023804,0.068176,0.119995,0.129639,0.128662,0.139038,0.144287,0.197876


- matrix에 월별 category별 평균 판매 수량(date_cat_avg_item_cnt) lag 추가하기

In [46]:
group = matrix4.groupby(['date_block_num', 'item_category_id']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_cat_avg_item_cnt']
group.reset_index(inplace=True)

matrix4 = pd.merge(matrix4, group, on=['date_block_num', 'item_category_id'], how='left')
matrix4['date_cat_avg_item_cnt'] = matrix4['date_cat_avg_item_cnt'].astype(np.float16)

matrix4 = lag_feature(matrix4, [1], 'date_cat_avg_item_cnt')
matrix4.drop(['date_cat_avg_item_cnt'], axis=1, inplace=True)
matrix4.tail()

,date_block_num,shop_id,item_id,ID,city_id,item_category_id,type_id,item_cnt_month,item_cnt_month_lag_1,item_cnt_month_lag_2,...,date_item_avg_item_cnt_lag_2,date_item_avg_item_cnt_lag_3,date_item_avg_item_cnt_lag_6,date_item_avg_item_cnt_lag_12,date_shop_avg_item_cnt_lag_1,date_shop_avg_item_cnt_lag_2,date_shop_avg_item_cnt_lag_3,date_shop_avg_item_cnt_lag_6,date_shop_avg_item_cnt_lag_12,date_cat_avg_item_cnt_lag_1
11127951,34,45,18454,214195,20,55,13,0.0,1.0,0.0,...,0.023254,0.071411,0.590820,NaN,0.129639,0.128662,0.139038,0.144287,NaN,0.197021
11127952,34,45,16188,214196,20,64,14,0.0,0.0,0.0,...,0.069763,NaN,NaN,NaN,0.129639,0.128662,NaN,NaN,NaN,0.155640
11127953,34,45,15757,214197,20,55,13,0.0,0.0,0.0,...,0.069763,0.095215,0.250000,0.180054,0.129639,0.128662,0.139038,0.144287,0.197876,0.197021
11127954,34,45,19648,214198,20,40,11,0.0,0.0,0.0,...,0.069763,0.166626,0.090881,NaN,0.129639,0.128662,0.139038,0.144287,NaN,0.221558
11127955,34,45,969,214199,20,37,11,0.0,0.0,0.0,...,0.116272,0.023804,0.068176,0.119995,0.129639,0.128662,0.139038,0.144287,0.197876,0.262939


- matrix에 월별 shop별, category별 평균 판매 수량(date_shop_cat_avg_item_cnt) lag 추가하기

In [49]:
group = matrix4.groupby(['date_block_num', 'shop_id', 'item_category_id']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_shop_cat_avg_item_cnt']
group.reset_index(inplace=True)

matrix4 = pd.merge(matrix4, group, on=['date_block_num', 'shop_id', 'item_category_id'], how='left')
matrix4['date_shop_cat_avg_item_cnt'] = matrix4['date_shop_cat_avg_item_cnt'].astype(np.float16)

matrix4 = lag_feature(matrix4, [1], 'date_shop_cat_avg_item_cnt')
matrix4.drop(['date_shop_cat_avg_item_cnt'], axis=1, inplace=True)

In [50]:
matrix4.tail()

,date_block_num,shop_id,item_id,ID,city_id,item_category_id,type_id,item_cnt_month,item_cnt_month_lag_1,item_cnt_month_lag_2,...,date_item_avg_item_cnt_lag_3,date_item_avg_item_cnt_lag_6,date_item_avg_item_cnt_lag_12,date_shop_avg_item_cnt_lag_1,date_shop_avg_item_cnt_lag_2,date_shop_avg_item_cnt_lag_3,date_shop_avg_item_cnt_lag_6,date_shop_avg_item_cnt_lag_12,date_cat_avg_item_cnt_lag_1,date_shop_cat_avg_item_cnt_lag_1
11127951,34,45,18454,214195,20,55,13,0.0,1.0,0.0,...,0.071411,0.590820,NaN,0.129639,0.128662,0.139038,0.144287,NaN,0.197021,0.126831
11127952,34,45,16188,214196,20,64,14,0.0,0.0,0.0,...,NaN,NaN,NaN,0.129639,0.128662,NaN,NaN,NaN,0.155640,0.094482
11127953,34,45,15757,214197,20,55,13,0.0,0.0,0.0,...,0.095215,0.250000,0.180054,0.129639,0.128662,0.139038,0.144287,0.197876,0.197021,0.126831
11127954,34,45,19648,214198,20,40,11,0.0,0.0,0.0,...,0.166626,0.090881,NaN,0.129639,0.128662,0.139038,0.144287,NaN,0.221558,0.083740
11127955,34,45,969,214199,20,37,11,0.0,0.0,0.0,...,0.023804,0.068176,0.119995,0.129639,0.128662,0.139038,0.144287,0.197876,0.262939,0.128174


- matrix에 월별 shop별, type_id별 평균 판매 수량(date_shop_type_avg_item_cnt) lag 추가하기

In [52]:
group = matrix4.groupby(['date_block_num', 'shop_id', 'type_id']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_shop_type_avg_item_cnt']
group.reset_index(inplace=True)

matrix4 = pd.merge(matrix4, group, on=['date_block_num', 'shop_id', 'type_id'], how='left')
matrix4['date_shop_type_avg_item_cnt'] = matrix4['date_shop_type_avg_item_cnt'].astype(np.float16)
matrix4 = lag_feature(matrix4, [1], 'date_shop_type_avg_item_cnt')
matrix4.drop(['date_shop_type_avg_item_cnt'], axis=1, inplace=True)
matrix4.tail()

,date_block_num,shop_id,item_id,ID,city_id,item_category_id,type_id,item_cnt_month,item_cnt_month_lag_1,item_cnt_month_lag_2,...,date_item_avg_item_cnt_lag_6,date_item_avg_item_cnt_lag_12,date_shop_avg_item_cnt_lag_1,date_shop_avg_item_cnt_lag_2,date_shop_avg_item_cnt_lag_3,date_shop_avg_item_cnt_lag_6,date_shop_avg_item_cnt_lag_12,date_cat_avg_item_cnt_lag_1,date_shop_cat_avg_item_cnt_lag_1,date_shop_type_avg_item_cnt_lag_1
11127951,34,45,18454,214195,20,55,13,0.0,1.0,0.0,...,0.590820,NaN,0.129639,0.128662,0.139038,0.144287,NaN,0.197021,0.126831,0.089294
11127952,34,45,16188,214196,20,64,14,0.0,0.0,0.0,...,NaN,NaN,0.129639,0.128662,NaN,NaN,NaN,0.155640,0.094482,0.112976
11127953,34,45,15757,214197,20,55,13,0.0,0.0,0.0,...,0.250000,0.180054,0.129639,0.128662,0.139038,0.144287,0.197876,0.197021,0.126831,0.089294
11127954,34,45,19648,214198,20,40,11,0.0,0.0,0.0,...,0.090881,NaN,0.129639,0.128662,0.139038,0.144287,NaN,0.221558,0.083740,0.097046
11127955,34,45,969,214199,20,37,11,0.0,0.0,0.0,...,0.068176,0.119995,0.129639,0.128662,0.139038,0.144287,0.197876,0.262939,0.128174,0.097046


- matrix에 월별 city_id별 평균 판매 수량(date_city_avg_item_cnt) lag 추가하기

In [53]:
group = matrix4.groupby(['date_block_num', 'city_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_city_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix4 = pd.merge(matrix4, group, on=['date_block_num', 'city_id'], how='left')
matrix4['date_city_avg_item_cnt'] = matrix4['date_city_avg_item_cnt'].astype(np.float16)
matrix4 = lag_feature(matrix4, [1], 'date_city_avg_item_cnt')
matrix4.drop(['date_city_avg_item_cnt'], axis=1, inplace=True)
matrix4.tail()

,date_block_num,shop_id,item_id,ID,city_id,item_category_id,type_id,item_cnt_month,item_cnt_month_lag_1,item_cnt_month_lag_2,...,date_item_avg_item_cnt_lag_12,date_shop_avg_item_cnt_lag_1,date_shop_avg_item_cnt_lag_2,date_shop_avg_item_cnt_lag_3,date_shop_avg_item_cnt_lag_6,date_shop_avg_item_cnt_lag_12,date_cat_avg_item_cnt_lag_1,date_shop_cat_avg_item_cnt_lag_1,date_shop_type_avg_item_cnt_lag_1,date_city_avg_item_cnt_lag_1
11127951,34,45,18454,214195,20,55,13,0.0,1.0,0.0,...,NaN,0.129639,0.128662,0.139038,0.144287,NaN,0.197021,0.126831,0.089294,0.136841
11127952,34,45,16188,214196,20,64,14,0.0,0.0,0.0,...,NaN,0.129639,0.128662,NaN,NaN,NaN,0.155640,0.094482,0.112976,0.136841
11127953,34,45,15757,214197,20,55,13,0.0,0.0,0.0,...,0.180054,0.129639,0.128662,0.139038,0.144287,0.197876,0.197021,0.126831,0.089294,0.136841
11127954,34,45,19648,214198,20,40,11,0.0,0.0,0.0,...,NaN,0.129639,0.128662,0.139038,0.144287,NaN,0.221558,0.083740,0.097046,0.136841
11127955,34,45,969,214199,20,37,11,0.0,0.0,0.0,...,0.119995,0.129639,0.128662,0.139038,0.144287,0.197876,0.262939,0.128174,0.097046,0.136841


- matrix에 월별 item별, city_id별 평균 판매 수량(date_item_city_avg_item_cnt) lag 추가하기

In [54]:
group = matrix4.groupby(['date_block_num', 'item_id', 'city_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_item_city_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix4 = pd.merge(matrix4, group, on=['date_block_num', 'item_id', 'city_id'], how='left')
matrix4['date_item_city_avg_item_cnt'] = matrix4['date_item_city_avg_item_cnt'].astype(np.float16)

matrix4 = lag_feature(matrix4, [1], 'date_item_city_avg_item_cnt')
matrix4.drop(['date_item_city_avg_item_cnt'], axis=1, inplace=True)
matrix4.tail()

,date_block_num,shop_id,item_id,ID,city_id,item_category_id,type_id,item_cnt_month,item_cnt_month_lag_1,item_cnt_month_lag_2,...,date_shop_avg_item_cnt_lag_1,date_shop_avg_item_cnt_lag_2,date_shop_avg_item_cnt_lag_3,date_shop_avg_item_cnt_lag_6,date_shop_avg_item_cnt_lag_12,date_cat_avg_item_cnt_lag_1,date_shop_cat_avg_item_cnt_lag_1,date_shop_type_avg_item_cnt_lag_1,date_city_avg_item_cnt_lag_1,date_item_city_avg_item_cnt_lag_1
11127951,34,45,18454,214195,20,55,13,0.0,1.0,0.0,...,0.129639,0.128662,0.139038,0.144287,NaN,0.197021,0.126831,0.089294,0.136841,0.5
11127952,34,45,16188,214196,20,64,14,0.0,0.0,0.0,...,0.129639,0.128662,NaN,NaN,NaN,0.155640,0.094482,0.112976,0.136841,0.0
11127953,34,45,15757,214197,20,55,13,0.0,0.0,0.0,...,0.129639,0.128662,0.139038,0.144287,0.197876,0.197021,0.126831,0.089294,0.136841,0.0
11127954,34,45,19648,214198,20,40,11,0.0,0.0,0.0,...,0.129639,0.128662,0.139038,0.144287,NaN,0.221558,0.083740,0.097046,0.136841,0.0
11127955,34,45,969,214199,20,37,11,0.0,0.0,0.0,...,0.129639,0.128662,0.139038,0.144287,0.197876,0.262939,0.128174,0.097046,0.136841,0.5


- matrix에 월별 type_id별 평균 판매 수량(date_type_avg_item_cnt) lag 추가하기

In [55]:
group = matrix4.groupby(['date_block_num', 'type_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_type_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix4 = pd.merge(matrix4, group, on=['date_block_num', 'type_id'], how='left')
matrix4['date_type_avg_item_cnt'] = matrix4['date_type_avg_item_cnt'].astype(np.float16)
matrix4 = lag_feature(matrix4, [1], 'date_type_avg_item_cnt')
matrix4.drop(['date_type_avg_item_cnt'], axis=1, inplace=True)
matrix4.tail()

,date_block_num,shop_id,item_id,ID,city_id,item_category_id,type_id,item_cnt_month,item_cnt_month_lag_1,item_cnt_month_lag_2,...,date_shop_avg_item_cnt_lag_2,date_shop_avg_item_cnt_lag_3,date_shop_avg_item_cnt_lag_6,date_shop_avg_item_cnt_lag_12,date_cat_avg_item_cnt_lag_1,date_shop_cat_avg_item_cnt_lag_1,date_shop_type_avg_item_cnt_lag_1,date_city_avg_item_cnt_lag_1,date_item_city_avg_item_cnt_lag_1,date_type_avg_item_cnt_lag_1
11127951,34,45,18454,214195,20,55,13,0.0,1.0,0.0,...,0.128662,0.139038,0.144287,NaN,0.197021,0.126831,0.089294,0.136841,0.5,0.147095
11127952,34,45,16188,214196,20,64,14,0.0,0.0,0.0,...,0.128662,NaN,NaN,NaN,0.155640,0.094482,0.112976,0.136841,0.0,0.313232
11127953,34,45,15757,214197,20,55,13,0.0,0.0,0.0,...,0.128662,0.139038,0.144287,0.197876,0.197021,0.126831,0.089294,0.136841,0.0,0.147095
11127954,34,45,19648,214198,20,40,11,0.0,0.0,0.0,...,0.128662,0.139038,0.144287,NaN,0.221558,0.083740,0.097046,0.136841,0.0,0.226318
11127955,34,45,969,214199,20,37,11,0.0,0.0,0.0,...,0.128662,0.139038,0.144287,0.197876,0.262939,0.128174,0.097046,0.136841,0.5,0.226318


In [56]:
matrix5 = matrix4.copy()
matrix5.head()

,date_block_num,shop_id,item_id,ID,city_id,item_category_id,type_id,item_cnt_month,item_cnt_month_lag_1,item_cnt_month_lag_2,...,date_shop_avg_item_cnt_lag_2,date_shop_avg_item_cnt_lag_3,date_shop_avg_item_cnt_lag_6,date_shop_avg_item_cnt_lag_12,date_cat_avg_item_cnt_lag_1,date_shop_cat_avg_item_cnt_lag_1,date_shop_type_avg_item_cnt_lag_1,date_city_avg_item_cnt_lag_1,date_item_city_avg_item_cnt_lag_1,date_type_avg_item_cnt_lag_1
0,0,0,19,0,29,40,11,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,27,0,29,19,5,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,28,0,29,30,8,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,29,0,29,23,5,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0,32,0,29,40,11,6.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
matrix5.to_csv('matrix5.csv', header=True, index=False)

In [ ]:
for i in k:
    if type(i) != list:
        group = soa.groupby(['date_block_num', i]).agg({'item_cnt_month': ['mean']})
        group.columns = [ 'date_shop_avg_item_cnt' ]
        print(group)
        
    elif type(i) == list:
        for j in i:
            group = soa.groupby(['date_block_num', j]).agg({'item_cnt_month': ['mean']})
            group.columns = [ 'date_shop_avg_item_cnt' ]
            print(group)
            
    else:
        print('ERROR',end = '=='*10)



k = ['date_block_num','item_id','shop_id','item_category_id',['shop_id','item_category_id'],'city_code',['item_id','city_code'],'type_code']
lag_list = [1,2,3,6,12]